性能比較  
これまでのモデルを検証データで性能比較

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm

import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42) # 乱数生成シード
cudnn.benchmark = True

# Grab a GPU if there is one
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cuda device: 0


In [2]:
fold = "../validation_raw_npy/"
xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}sampled_label.npy")
print(xyz.shape)
print(label.shape)


(28789, 3, 500)
(28789,)


# v5

In [3]:
class ConvBNReLU_v5(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

            kernel_size=3,
            stride=1,
            padding=1,
            bias=True,
            dropout=0.0 # ドロップアウト率を新規追加
    ):
        super(ConvBNReLU_v5, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )
        if dropout > 0:
            self.main.append(nn.Dropout(dropout))

        # He初期化
        nn.init.kaiming_normal_(self.main[0].weight, mode='fan_out', nonlinearity='relu')
        if self.main[0].bias is not None:
            nn.init.constant_(self.main[0].bias, 0)
    
    def forward(self,x):
        return self.main(x)
    

    
class CNN_v5(nn.Module):
    def __init__(self, output_size=8, in_channels=3, num_filters_init=8, dropout=0.0):
        super(CNN_v5, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU_v5(in_channels, num_filters_init, 3,1,1), # 500 -> 500
            

            ConvBNReLU_v5(num_filters_init, num_filters_init*2, 8,2,3,dropout=dropout ), # 500->250
            
            ConvBNReLU_v5(num_filters_init*2, num_filters_init*4, 6,2,2,dropout=dropout), # 250 -> 125 (248)
            ConvBNReLU_v5(num_filters_init*4, num_filters_init*8,7,2,2, dropout=dropout), # 125 -> 62
            ConvBNReLU_v5(num_filters_init*8, num_filters_init*16,6,2,1 , dropout=dropout), # 62 -> 30

            ConvBNReLU_v5(num_filters_init*16, num_filters_init*32, 4, 2, 1, dropout=dropout), # 30 -> 15
            ConvBNReLU_v5(num_filters_init*32, num_filters_init*64, 3, 2, 1, dropout=dropout), # 15->8

            ConvBNReLU_v5(num_filters_init*64, num_filters_init*128, 8,1,0), # 8 -> 1 
            #nn.Conv1d(num_filters_init*128, output_size, 1,1,0) #代わりに下の全結合層を導入
            ConvBNReLU_v5(num_filters_init*128, num_filters_init*64, 1,1,0, dropout=dropout),
            ConvBNReLU_v5(num_filters_init*64, 128, 1,1,0,dropout=dropout),
            ConvBNReLU_v5(128, output_size, 1,1,0),

        )
        # 全結合層
        """self.fc = nn.Sequential(
            nn.Linear(num_filters_init*128, num_filters_init*64),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(num_filters_init*64, 128),
            nn.ReLU(True),
            nn.Dropout(dropout),

            nn.Linear(128, output_size)
        )"""

        #nn.init.kaiming_normal_(self.fc[0].weight, mode="fan_out", nonlinearity="relu")
        #nn.init.kaiming_normal_(self.fc[3].weight, mode="fan_out", nonlinearity="relu")
        

        # 最後のConv1dにもHe初期化を適用
        #nn.init.kaiming_normal_(self.cnn[-1].weight, mode='fan_out', nonlinearity='relu')
        #if self.cnn[-1].bias is not None:
        #    nn.init.constant_(self.cnn[-1].bias, 0)
        
    
    def forward(self, x):
        x = self.cnn(x).view(x.size(0), -1)
        #return self.fc(x)
        return x

In [4]:
def create_dataloader_v5(X, y=None, batch_size=1, shuffle=False):
    if shuffle:
        idxs = np.random.permutation(np.arange(len(X)))
    else:
        idxs = np.arange(len(X))
    
    #データセットをバッチサイズごとに分割
    for i in range(0, len(idxs), batch_size):
        idxs_batch = idxs[i:i+batch_size]
        X_batch = X[idxs_batch].astype('f4')

        X_batch = torch.from_numpy(X_batch)
        if y is None:
            yield X_batch
        else:
            y_batch = y[idxs_batch]
            y_batch = torch.from_numpy(y_batch-1)
            yield X_batch, y_batch


def forward_by_batches_v5(cnn, X):
    Y = []
    cnn.eval()
    with torch.no_grad():
        for x in create_dataloader_v5(X, batch_size=1024, shuffle=False):
            x = x.to(device)
            Y.append(cnn(x))
    cnn.train()

    Y = torch.cat(Y) # Yをテンソルに変換
    return Y


def evaluate_model_v5(cnn, X, Y):
    Y_pred = forward_by_batches_v5(cnn, X)
    loss = F.cross_entropy(Y_pred, torch.from_numpy(Y-1.0).type(torch.int64).to(device)).item() # クロスエントロピー損失の計算 仕方なく0~7クラス分類とする

    Y_pred = F.softmax(Y_pred, dim=1) 
    Y_pred = torch.argmax(Y_pred, dim=1)  # 最も高い確率のY_predのラベルを予測ラベルとしてY_predに入れられる。
    Y_pred = Y_pred + 1  # 予測ラベルに1を加えて1~8の範囲に変換する
    Y_pred = Y_pred.cpu().numpy()  # テンソルでGPUにのっているものをcpuに移動して、それをさらにnumpy配列に変換している。
    kappa = metrics.cohen_kappa_score(Y, Y_pred) # 1~8クラス分類

    return {'loss':loss, 'kappa':kappa, 'Y_pred':Y_pred}

In [5]:
num_filters_init = 32
in_channels = 3  
output_size = len(np.unique(label))  #　これは分類するラベルの数。最終層の特徴量の数
num_epoch = 5  # num of epochs (full loops though the training set)
lr = 1e-3  # learning rate 3e-4だった


model_save_path = "../models/cnn_model_v5.pth"
cnn_v5 = CNN_v5(
    output_size=output_size,
    in_channels=in_channels,
    num_filters_init=num_filters_init,
    dropout=0.35
).to(device)
cnn_v5.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [6]:
cnn_v5.eval()

CNN_v5(
  (cnn): Sequential(
    (0): ConvBNReLU_v5(
      (main): Sequential(
        (0): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBNReLU_v5(
      (main): Sequential(
        (0): Conv1d(32, 64, kernel_size=(8,), stride=(2,), padding=(3,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.35, inplace=False)
      )
    )
    (2): ConvBNReLU_v5(
      (main): Sequential(
        (0): Conv1d(64, 128, kernel_size=(6,), stride=(2,), padding=(2,))
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.35, inplace=False)
      )
    )
    (3): ConvBNReLU_v5(
      (main): Sequential(
        (0): Conv1d(128, 256, kernel_size=(7,)

# v4

In [7]:
class ConvBNReLU_v4(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

            kernel_size=3,
            stride=1,
            padding=1,
            bias=True,
            dropout=0.0 # ドロップアウト率を新規追加
    ):
        super(ConvBNReLU_v4, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )
        if dropout > 0:
            self.main.append(nn.Dropout(dropout))

        # He初期化
        nn.init.kaiming_normal_(self.main[0].weight, mode='fan_out', nonlinearity='relu')
        if self.main[0].bias is not None:
            nn.init.constant_(self.main[0].bias, 0)
    
    def forward(self,x):
        return self.main(x)
    

    
class CNN_v4(nn.Module):
    def __init__(self, output_size=8, in_channels=3, num_filters_init=8, dropout=0.0):
        super(CNN_v4, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU_v4(in_channels, num_filters_init, 3,1,1), # 500 -> 500
            

            ConvBNReLU_v4(num_filters_init, num_filters_init*2, 8,2,3,dropout=dropout ), # 500->250
            
            ConvBNReLU_v4(num_filters_init*2, num_filters_init*4, 6,2,2,dropout=dropout), # 250 -> 125 (248)
            ConvBNReLU_v4(num_filters_init*4, num_filters_init*8,7,2,2, dropout=dropout), # 125 -> 62
            ConvBNReLU_v4(num_filters_init*8, num_filters_init*16,6,2,1 , dropout=dropout), # 62 -> 30

            ConvBNReLU_v4(num_filters_init*16, num_filters_init*32, 4, 2, 1, dropout=dropout), # 30 -> 15
            ConvBNReLU_v4(num_filters_init*32, num_filters_init*64, 3, 2, 1, dropout=dropout), # 15->8

            ConvBNReLU_v4(num_filters_init*64, num_filters_init*128, 8,1,0), # 8 -> 1 
            #nn.Conv1d(num_filters_init*128, output_size, 1,1,0) #代わりに下の全結合層を導入
        )
        # 全結合層
        self.fc = nn.Sequential(
            nn.Linear(num_filters_init*128, num_filters_init*64),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(num_filters_init*64, 128),
            nn.ReLU(True),
            nn.Dropout(dropout),

            nn.Linear(128, output_size)
        )

        nn.init.kaiming_normal_(self.fc[0].weight, mode="fan_out", nonlinearity="relu")
        nn.init.kaiming_normal_(self.fc[3].weight, mode="fan_out", nonlinearity="relu")
        

        # 最後のConv1dにもHe初期化を適用
        #nn.init.kaiming_normal_(self.cnn[-1].weight, mode='fan_out', nonlinearity='relu')
        #if self.cnn[-1].bias is not None:
        #    nn.init.constant_(self.cnn[-1].bias, 0)
        
    
    def forward(self, x):
        x = self.cnn(x).view(x.size(0), -1)
        return self.fc(x)
        #return x

In [8]:
def create_dataloader_v4(X, y=None, batch_size=1, shuffle=False):
    if shuffle:
        idxs = np.random.permutation(np.arange(len(X)))
    else:
        idxs = np.arange(len(X))
    
    #データセットをバッチサイズごとに分割
    for i in range(0, len(idxs), batch_size):
        idxs_batch = idxs[i:i+batch_size]
        X_batch = X[idxs_batch].astype('f4')

        X_batch = torch.from_numpy(X_batch)
        if y is None:
            yield X_batch
        else:
            y_batch = y[idxs_batch]
            y_batch = torch.from_numpy(y_batch-1)
            yield X_batch, y_batch


def forward_by_batches_v4(cnn, X):
    Y = []
    cnn.eval()
    with torch.no_grad():
        for x in create_dataloader_v4(X, batch_size=1024, shuffle=False):
            x = x.to(device)
            Y.append(cnn(x))
    cnn.train()

    Y = torch.cat(Y) # Yをテンソルに変換
    return Y


def evaluate_model_v4(cnn, X, Y):
    Y_pred = forward_by_batches_v4(cnn, X)
    loss = F.cross_entropy(Y_pred, torch.from_numpy(Y-1.0).type(torch.int64).to(device)).item() # クロスエントロピー損失の計算 仕方なく0~7クラス分類とする

    Y_pred = F.softmax(Y_pred, dim=1) 
    Y_pred = torch.argmax(Y_pred, dim=1)  # 最も高い確率のY_predのラベルを予測ラベルとしてY_predに入れられる。
    Y_pred = Y_pred + 1  # 予測ラベルに1を加えて1~8の範囲に変換する
    Y_pred = Y_pred.cpu().numpy()  # テンソルでGPUにのっているものをcpuに移動して、それをさらにnumpy配列に変換している。
    kappa = metrics.cohen_kappa_score(Y, Y_pred) # 1~8クラス分類

    return {'loss':loss, 'kappa':kappa, 'Y_pred':Y_pred}

In [9]:
import torch.optim.lr_scheduler as lr_scheduler
num_filters_init = 32
in_channels = 3  
output_size = len(np.unique(label))  #　これは分類するラベルの数。最終層の特徴量の数
num_epoch = 5  # num of epochs (full loops though the training set)
lr = 1e-3  # learning rate 3e-4だった

model_save_path = "../models/cnn_model_v4.pth"
cnn_v4 = CNN_v4(
    output_size=output_size,
    in_channels=in_channels,
    num_filters_init=num_filters_init,
    dropout=0.35
).to(device)
cnn_v4.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [10]:
cnn_v4.eval()

CNN_v4(
  (cnn): Sequential(
    (0): ConvBNReLU_v4(
      (main): Sequential(
        (0): Conv1d(3, 32, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBNReLU_v4(
      (main): Sequential(
        (0): Conv1d(32, 64, kernel_size=(8,), stride=(2,), padding=(3,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.35, inplace=False)
      )
    )
    (2): ConvBNReLU_v4(
      (main): Sequential(
        (0): Conv1d(64, 128, kernel_size=(6,), stride=(2,), padding=(2,))
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.35, inplace=False)
      )
    )
    (3): ConvBNReLU_v4(
      (main): Sequential(
        (0): Conv1d(128, 256, kernel_size=(7,)

# v7

In [11]:
class ConvBNReLU_v7(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,

            kernel_size=3,
            stride=1,
            padding=1,
            bias=True,
            dropout=0.0 # ドロップアウト率を新規追加
    ):
        super(ConvBNReLU_v7, self).__init__()

        self.main = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=bias),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(True)
        )
        if dropout > 0:
            self.main.append(nn.Dropout(dropout))

        # He初期化
        nn.init.kaiming_normal_(self.main[0].weight, mode='fan_out', nonlinearity='relu')
        if self.main[0].bias is not None:
            nn.init.constant_(self.main[0].bias, 0)
    
    def forward(self,x):
        return self.main(x)
    

    
class CNN_v7(nn.Module):
    def __init__(self, output_size=8, in_channels=3, num_filters_init=8, dropout=0.0):
        super(CNN_v7, self).__init__()

        self.cnn = nn.Sequential(
            ConvBNReLU_v7(in_channels, num_filters_init, 3,1,1), # 500 -> 500
            ConvBNReLU_v7(num_filters_init, num_filters_init*2, 8,2,3,dropout=dropout ), # 500->250
            ConvBNReLU_v7(num_filters_init*2, num_filters_init*3, 6,2,2,dropout=dropout), # 250 -> 125 (248)
            ConvBNReLU_v7(num_filters_init*3, num_filters_init*3,7,2,2, dropout=dropout), # 125 -> 62
            ConvBNReLU_v7(num_filters_init*3, num_filters_init*4,6,2,1 , dropout=dropout), # 62 -> 30
            ConvBNReLU_v7(num_filters_init*4, num_filters_init*4, 4, 2, 1, dropout=dropout), # 30 -> 15
            ConvBNReLU_v7(num_filters_init*4, num_filters_init*4, 3, 2, 1, dropout=dropout), # 15->8
            ConvBNReLU_v7(num_filters_init*4, num_filters_init*4, 8,1,0, dropout=dropout), # 8 -> 1 
            #nn.Conv1d(num_filters_init*128, output_size, 1,1,0) #代わりに下の全結合層を導入
        )
        # 全結合層
        self.fc = nn.Sequential(
            nn.Linear(num_filters_init*4, num_filters_init*2),
            nn.ReLU(True),
            nn.Dropout(dropout),
            nn.Linear(num_filters_init*2, num_filters_init),
            nn.ReLU(True),
            nn.Dropout(dropout),
            
            nn.Linear(num_filters_init, output_size)
        )

        nn.init.kaiming_normal_(self.fc[0].weight, mode="fan_out", nonlinearity="relu")
        nn.init.kaiming_normal_(self.fc[3].weight, mode="fan_out", nonlinearity="relu")
        #nn.init.kaiming_normal_(self.fc[6].weight, mode="fan_out", nonlinearity="relu")
        
        

        # 最後のConv1dにもHe初期化を適用
        #nn.init.kaiming_normal_(self.cnn[-1].weight, mode='fan_out', nonlinearity='relu')
        #if self.cnn[-1].bias is not None:
        #    nn.init.constant_(self.cnn[-1].bias, 0)
        
    
    def forward(self, x):
        x = self.cnn(x).view(x.size(0), -1)
        return self.fc(x)
        #return x

In [12]:
def create_dataloader_v7(X, y=None, batch_size=1, shuffle=False):
    if shuffle:
        idxs = np.random.permutation(np.arange(len(X)))
    else:
        idxs = np.arange(len(X))
    
    #データセットをバッチサイズごとに分割
    for i in range(0, len(idxs), batch_size):
        idxs_batch = idxs[i:i+batch_size]
        X_batch = X[idxs_batch].astype('f4')

        X_batch = torch.from_numpy(X_batch)
        if y is None:
            yield X_batch
        else:
            y_batch = y[idxs_batch]
            y_batch = torch.from_numpy(y_batch-1)
            yield X_batch, y_batch


def forward_by_batches_v7(cnn, X):
    Y = []
    cnn.eval()
    with torch.no_grad():
        for x in create_dataloader_v7(X, batch_size=1024, shuffle=False):
            x = x.to(device)
            Y.append(cnn(x))
    cnn.train()

    Y = torch.cat(Y) # Yをテンソルに変換
    return Y


def evaluate_model_v7(cnn, X, Y):
    Y_pred = forward_by_batches_v7(cnn, X)
    loss = F.cross_entropy(Y_pred, torch.from_numpy(Y-1.0).type(torch.int64).to(device)).item() # クロスエントロピー損失の計算 仕方なく0~7クラス分類とする

    Y_pred = F.softmax(Y_pred, dim=1) 
    Y_pred = torch.argmax(Y_pred, dim=1)  # 最も高い確率のY_predのラベルを予測ラベルとしてY_predに入れられる。
    Y_pred = Y_pred + 1  # 予測ラベルに1を加えて1~8の範囲に変換する
    Y_pred = Y_pred.cpu().numpy()  # テンソルでGPUにのっているものをcpuに移動して、それをさらにnumpy配列に変換している。
    kappa = metrics.cohen_kappa_score(Y, Y_pred) # 1~8クラス分類

    return {'loss':loss, 'kappa':kappa, 'Y_pred':Y_pred}

In [13]:
import torch.optim.lr_scheduler as lr_scheduler
num_filters_init = 25 # 16:    32:0.85(test limit)
in_channels = 3  
output_size = len(np.unique(label))  #　これは分類するラベルの数。最終層の特徴量の数
num_epoch = 5  # num of epochs (full loops though the training set)
lr = 1e-3  # learning rate 3e-4だった


model_save_path = "../models/cnn_v7_all.pth"
cnn_v7 = CNN_v7(
    output_size=output_size,
    in_channels=in_channels,
    num_filters_init=num_filters_init,
    dropout=0.2 # filter=32, dropout0.3
).to(device)
cnn_v7.load_state_dict(torch.load(model_save_path))

<All keys matched successfully>

In [14]:
cnn_v7.eval()

CNN_v7(
  (cnn): Sequential(
    (0): ConvBNReLU_v7(
      (main): Sequential(
        (0): Conv1d(3, 25, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBNReLU_v7(
      (main): Sequential(
        (0): Conv1d(25, 50, kernel_size=(8,), stride=(2,), padding=(3,))
        (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (2): ConvBNReLU_v7(
      (main): Sequential(
        (0): Conv1d(50, 75, kernel_size=(6,), stride=(2,), padding=(2,))
        (1): BatchNorm1d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (3): ConvBNReLU_v7(
      (main): Sequential(
        (0): Conv1d(75, 75, kernel_size=(7,), stri

In [15]:
result_v4 = evaluate_model_v4(cnn_v4, xyz, label)
result_v5 = evaluate_model_v5(cnn_v5, xyz, label)
result_v7 = evaluate_model_v7(cnn_v7, xyz, label)

In [16]:
print('\nClassifier performance v4')
print('Out of sample:\n', metrics.classification_report(label, result_v4['Y_pred']))
print('\nClassifier performance v5')
print('Out of sample:\n', metrics.classification_report(label, result_v5['Y_pred']))
print('\nClassifier performance v7')
print('Out of sample:\n', metrics.classification_report(label, result_v7['Y_pred']))


Classifier performance v4
Out of sample:
               precision    recall  f1-score   support

         1.0       0.27      0.91      0.41      5967
         2.0       0.81      0.23      0.36      5225
         3.0       0.97      0.52      0.67       555
         4.0       0.92      0.26      0.41      2407
         5.0       0.10      0.01      0.03      4095
         6.0       0.18      0.11      0.14      1836
         7.0       0.20      0.18      0.19      4362
         8.0       0.25      0.02      0.03      4342

    accuracy                           0.30     28789
   macro avg       0.46      0.28      0.28     28789
weighted avg       0.39      0.30      0.24     28789


Classifier performance v5
Out of sample:
               precision    recall  f1-score   support

         1.0       0.26      0.93      0.40      5967
         2.0       0.89      0.35      0.51      5225
         3.0       0.96      0.54      0.69       555
         4.0       0.95      0.29      0.45   